In [2]:
# Importing libraries
import pandas as pd
import requests
import io
import warnings
import time

start = time.time()

# Set warnings off
warnings.filterwarnings("ignore")

Data Cleaning

In [3]:
df = pd.read_csv('Data_FeatureFinish.csv')

In [4]:
df.head()

,Fecha,Hora,Clase de Vuelo,Clasificación Vuelo,Tipo Movimiento,Aeropuerto,Origen/Destino,Aerolinea Nombre,Aeronave,Pasajeros,PAX,Calidad dato
0,1/1/2019,00:01:00,REGULAR,Doméstico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,88,44,DEFINITIVO
1,1/1/2019,00:01:00,REGULAR,Internacional,Aterrizaje,EZE,SBGL,TRANSPORTES AEREOS DEL MERCOSUR,NaN,165,165,DEFINITIVO
2,1/1/2019,00:03:00,REGULAR,Doméstico,Aterrizaje,AER,SIS,AUSTRAL LINEAS AEREAS-CIELOS DEL SUR S.A,EMB-ERJ190100IGW,22,11,DEFINITIVO
3,1/1/2019,00:04:00,REGULAR,Internacional,Aterrizaje,EZE,SBGR,TRANSPORTES AEREOS DEL MERCOSUR,NaN,69,69,DEFINITIVO
4,1/1/2019,00:06:00,REGULAR,Internacional,Aterrizaje,AER,SBGR,LAN ARGENTINA S.A. (LATAM AIRLINES),AIB-A-320-233,53,53,DEFINITIVO


In [5]:
df.columns

Index(['Fecha', 'Hora', 'Clase de Vuelo', 'Clasificación Vuelo',
       'Tipo Movimiento', 'Aeropuerto', 'Origen/Destino', 'Aerolinea Nombre',
       'Aeronave', 'Pasajeros', 'PAX', 'Calidad dato'],
      dtype='object')

In [6]:
# Columns to analyze
columns = ['Clase de Vuelo', 'Clasificación Vuelo', 'Tipo Movimiento', 'Calidad dato']

# Checking the unique values of each column
for column in columns:
    print(f'Unique values for the column: {column}')
    print(df[column].unique())

    print('\n************************************************************************\n')

Unique values for the column: Clase de Vuelo
['REGULAR' 'VUELOS PRIVADOS NACIONALES' 'VUELOS OFICIALES NACIONALES'
 'NO REGULAR' 'VUELOS PRIVADO CON MATRICULA EXTRANJERA' 'TRABAJO AEREO'
 'VUELOS DE ADIESTRAMIENTO' 'VUELOS ESCUELA'
 'VUELOS OFICIALES EXTRANJEROS' 'ESCUELA (NO VIGENTE)' 'Regular'
 'Vuelo Privado con Matrícula Nacional' 'No Regular'
 'Vuelo Privado con Matrícula Extranjera' 'Trabajo Aéreo'
 'Vuelo de Adiestramiento' 'Vuelo Oficial Extranjero'
 'Vuelo Oficial Nacional' 'Vuelo Escuela']

************************************************************************

Unique values for the column: Clasificación Vuelo
['Doméstico' 'Internacional' 'Dom' 'Inter']

************************************************************************

Unique values for the column: Tipo Movimiento
['Aterrizaje' 'Despegue' nan]

************************************************************************

Unique values for the column: Calidad dato
['DEFINITIVO' 'PROVISORIO']

****************************

In [7]:
# Recoding the values of column 'Clase de Vuelo'
df['Clase de Vuelo'] = df['Clase de Vuelo'].str.capitalize()

In [8]:
df['Clase de Vuelo'].unique()

array(['Regular', 'Vuelos privados nacionales',
       'Vuelos oficiales nacionales', 'No regular',
       'Vuelos privado con matricula extranjera', 'Trabajo aereo',
       'Vuelos de adiestramiento', 'Vuelos escuela',
       'Vuelos oficiales extranjeros', 'Escuela (no vigente)',
       'Vuelo privado con matrícula nacional',
       'Vuelo privado con matrícula extranjera', 'Trabajo aéreo',
       'Vuelo de adiestramiento', 'Vuelo oficial extranjero',
       'Vuelo oficial nacional', 'Vuelo escuela'], dtype=object)

In [9]:
# Define how the values are going to change
fligth_class = {
    'Vuelos privados nacionales': 'Vuelo privado nacional',
    'Vuelos oficiales nacionales': 'Vuelo oficial nacional',
    'Vuelos privado con matricula extranjera': 'Vuelo privado con matrícula extranjera',
    'Trabajo aereo': 'Trabajo aéreo',
    'Vuelos de adiestramiento': 'Vuelo de adiestramiento',
    'Vuelos escuela': 'Vuelo escuela',
    'Vuelos oficiales extranjeros': 'Vuelo oficial extranjero'
}

# Replacing the values
df['Clase de Vuelo'].replace(fligth_class, inplace=True)

In [10]:
# Replacing the values of column 'Clasificación Vuelo'
df['Clasificación Vuelo'] = df['Clasificación Vuelo'].replace({
    'Dom': 'Doméstico',
    'Inter': 'Internacional'
})

In [11]:
# Checking the unique values of each column
for column in columns:
    print(f'Unique values for the column: {column}')
    print(df[column].unique())

    print('\n************************************************************************\n')

Unique values for the column: Clase de Vuelo
['Regular' 'Vuelo privado nacional' 'Vuelo oficial nacional' 'No regular'
 'Vuelo privado con matrícula extranjera' 'Trabajo aéreo'
 'Vuelo de adiestramiento' 'Vuelo escuela' 'Vuelo oficial extranjero'
 'Escuela (no vigente)' 'Vuelo privado con matrícula nacional']

************************************************************************

Unique values for the column: Clasificación Vuelo
['Doméstico' 'Internacional']

************************************************************************

Unique values for the column: Tipo Movimiento
['Aterrizaje' 'Despegue' nan]

************************************************************************

Unique values for the column: Calidad dato
['DEFINITIVO' 'PROVISORIO']

************************************************************************



In [12]:
data_columns = []

for column in columns:
    data_column = df.groupby(by=column).size().reset_index().sort_values(by=0, ascending=False).rename(columns={0:'Quantity'})
    data_columns.append(data_column)

In [13]:
data_columns[0]

,Clase de Vuelo,Quantity
2,Regular,1162018
5,Vuelo escuela,289055
9,Vuelo privado con matrícula nacional,272590
1,No regular,173950
7,Vuelo oficial nacional,139675
10,Vuelo privado nacional,72277
4,Vuelo de adiestramiento,19335
8,Vuelo privado con matrícula extranjera,16722
3,Trabajo aéreo,8724
6,Vuelo oficial extranjero,1526


In [14]:
# Delete the register with the value 'Escuela (no vigente)'
df = df[df['Clase de Vuelo'] != 'Escuela (no vigente)']

In [15]:
data_columns = []

for column in columns:
    data_column = df.groupby(by=column).size().reset_index().sort_values(by=0, ascending=False).rename(columns={0:'Quantity'})
    data_columns.append(data_column)

In [16]:
# Checking the frequency of each value from the column 'Clase de Vuelo'
data_columns[0]

,Clase de Vuelo,Quantity
1,Regular,1162018
4,Vuelo escuela,289055
8,Vuelo privado con matrícula nacional,272590
0,No regular,173950
6,Vuelo oficial nacional,139675
9,Vuelo privado nacional,72277
3,Vuelo de adiestramiento,19335
7,Vuelo privado con matrícula extranjera,16722
2,Trabajo aéreo,8724
5,Vuelo oficial extranjero,1526


In [17]:
# Checking the values for the column 'Clasificación Vuelo'
data_columns[1]

,Clasificación Vuelo,Quantity
0,Doméstico,1799863
1,Internacional,356009


In [18]:
# Checking the values for the column 'Tipo Movimiento'
data_columns[2]

,Tipo Movimiento,Quantity
1,Despegue,1086137
0,Aterrizaje,1069732


In [19]:
# Checking the values for the column 'Calidad dato'
data_columns[3]

,Calidad dato,Quantity
0,DEFINITIVO,2075202
1,PROVISORIO,80670


Fixing NaN Values

In [20]:
df.isna().sum()

Fecha                       0
Hora                        0
Clase de Vuelo              0
Clasificación Vuelo         0
Tipo Movimiento             3
Aeropuerto                  0
Origen/Destino              3
Aerolinea Nombre       182406
Aeronave                91661
Pasajeros                   0
PAX                         0
Calidad dato                0
dtype: int64

In [21]:
# Delete NaN Values from 'Tipo Movimiento' and 'Origen/Destino' which are the same registers.
df = df[~df['Tipo Movimiento'].isna()]

In [22]:
df.isna().sum()

Fecha                       0
Hora                        0
Clase de Vuelo              0
Clasificación Vuelo         0
Tipo Movimiento             0
Aeropuerto                  0
Origen/Destino              0
Aerolinea Nombre       182404
Aeronave                91661
Pasajeros                   0
PAX                         0
Calidad dato                0
dtype: int64

In [55]:
# To fix the NaN Values of column 'Aerolinea Nombre' I am going to catch the 'Aerolinea Name' repeated the most for each 'Aeronave' value, and then fill the missing values on the column 'Aerolinea Nombre' with those.

# Catching the 'Aeronave' values only where 'Aerolinea Nombre' and 'Aeronave' have real values
plane_name = set(df[(~df['Aerolinea Nombre'].isna()) & (df['Aerolinea Nombre'] != '0') & (~df['Aeronave'].isna()) & (df['Aeronave'] != '0')]['Aeronave'])
plane_name.remove(0)

df_planes = df[(~df['Aerolinea Nombre'].isna()) & (df['Aerolinea Nombre'] != '0')]

setup_planes = {}
for plane in plane_name:
    data_reduce = df_planes[df_planes['Aeronave'] == plane].groupby(by='Aerolinea Nombre').size().reset_index().rename(columns={0:'Quantity'})
    max_value = data_reduce['Quantity'].idxmax()
    max_value = data_reduce.loc[max_value, 'Aerolinea Nombre']
    setup_planes[plane] = max_value

print(setup_planes)

{'PA-PA-28-180': 'HANGAR UNO', 'BE-35': 'AEROLINEAS ARGENTINAS SA', 'AEP-ATR-42-320': 'AMERICAN JET S.A.', 'CE-421-A': 'AAXOD S.A', 'EL-206-B': 'SOCORRO MEDICO PRIVADO (VITTAL)', 'PA-PA-31-350': 'BROKER AIR S.A.', 'BE-E-90': 'SOCORRO MEDICO PRIVADO (VITTAL)', 'BE-C-35': 'AEROLINEAS ARGENTINAS SA', 'FA-SA-227-DC': 'AMERICAN JET S.A.', 'CE-172-P': 'AEROLINEAS ARGENTINAS SA', 'CE-T-188-C': 'AEROLINEAS ARGENTINAS SA', 'RO-R66': 'LA TITI S.A.', 'TEA-P92 ECHO CLA': 'VOLAR S.R.L', 'CE-182-G': 'D.G.A.C. SANTIAGO DEL ESTERO', 'HU-269-C': 'AEROLINEAS ARGENTINAS SA', 'TSA-S2R-T34': 'AEROLINEAS ARGENTINAS SA', 'CE-182': 'AEROLINEAS ARGENTINAS SA', 'CE-AA-150-L': 'FLIGHT LINE S.R.L.', 'CE-152-A': 'TWR SRL', 'CE-R-182': 'AEROLINEAS ARGENTINAS SA', 'LAN-IV-P': 'AEROTAX SOCIEDAD ANONIMA (PARAGUAY)', 'BE-58': 'AEROLINEAS ARGENTINAS SA', 'PAA-PA-A-28R201T': 'ECHO VICTOR', 'TXA-G58': 'AVIASUR CHARTER SERVICES S.A.', 'CE-180-A': 'AEROLINEAS ARGENTINAS SA', 'PA-PA-24-180': 'AEROLINEAS ARGENTINAS SA', 'LJ-2

In [ ]:
df[(df['Aeronave'] == '0') & (df['Aerolinea Nombre'] == '0')]

,Fecha,Hora,Clase de Vuelo,Clasificación Vuelo,Tipo Movimiento,Aeropuerto,Origen/Destino,Aerolinea Nombre,Aeronave,Pasajeros,PAX,Calidad dato
580904,1/1/2020,10:14,Vuelo privado con matrícula nacional,Internacional,Despegue,POS,2463,0,0,0,0,DEFINITIVO
580915,1/1/2020,10:35,Vuelo privado con matrícula nacional,Doméstico,Despegue,TUC,CBA,0,0,6,3,DEFINITIVO
580932,1/1/2020,11:06,Vuelo privado con matrícula nacional,Doméstico,Despegue,GAL,FDO,0,0,4,2,DEFINITIVO
580936,1/1/2020,11:12,No regular,Doméstico,Aterrizaje,BAR,LNH,0,0,2,1,DEFINITIVO
580959,1/1/2020,11:37,No regular,Doméstico,Despegue,BAR,LNH,0,0,2,1,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...,...
2153228,30/12/2023,2:50,Vuelo privado con matrícula extranjera,Internacional,Despegue,FDO,SULS,0,0,9,9,DEFINITIVO
2153252,30/12/2023,3:33,Vuelo privado con matrícula extranjera,Internacional,Despegue,FDO,SEGU,0,0,7,7,DEFINITIVO
2153296,30/12/2023,5:28,Vuelo privado con matrícula extranjera,Internacional,Aterrizaje,FDO,SULS,0,0,9,9,DEFINITIVO
2154916,31/12/2023,9:00,Vuelo oficial nacional,Doméstico,Despegue,CRV,CRV,0,0,0,0,DEFINITIVO


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2155872 entries, 0 to 2155872
Data columns (total 12 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Fecha                object
 1   Hora                 object
 2   Clase de Vuelo       object
 3   Clasificación Vuelo  object
 4   Tipo Movimiento      object
 5   Aeropuerto           object
 6   Origen/Destino       object
 7   Aerolinea Nombre     object
 8   Aeronave             object
 9   Pasajeros            int64 
 10  PAX                  object
 11  Calidad dato         object
dtypes: int64(1), object(11)
memory usage: 213.8+ MB
